# Extracting events given location and wind thresholds

This code does stuff...

By Claire Krause, October 2017. Python v2. TCRM branch `develop` (but should be applicable to other branches).

## Set up the notebook imports
Make sure the the TCRM directory is in the python path. A number of the import modules (e.g. `database`, `Utilities.`) are directly from TCRM.

In [1]:
%matplotlib inline
from __future__ import print_function, division

In [2]:
import database
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
import statsmodels.api as sm
from math import cos, asin, sqrt

#from windrose import WindroseAxes, WindAxes

from scipy.stats import genpareto, scoreatpercentile, lognorm

from Utilities.config import ConfigParser
from Utilities.lmomentFit import samlmu, pelgev, pelgpa
#from return_period import returnLevels, empiricalReturnPeriod, returnPeriodUncertainty
#from distributions import fittedPDF

# Import widgets for interactive notebook
from ipywidgets import interact, fixed
import ipywidgets as widgets

import seaborn as sns
sns.set_context("poster")
sns.set_style("whitegrid")

## Set up the hazard database instance
Point to the configuration file used to produce the database

In [3]:
configFile = "C:/Users/u37443/TCRM/output/port_hedland/Bodgy_config.ini"
config = ConfigParser()
config.read(configFile)

db = database.HazardDatabase(configFile)
locations = db.getLocations()
locNameList = list(locations['locName'])

No handlers could be found for logger "root"


## Set up the location/s that you would like to analyse

In [4]:
locations_to_analyse = {'Broome': [-17.952, 122.232], 'Port Hedland': [-20.310, 118.591], 
                        'Dampier/Karratha': [-20.704, 116.767], 'Exmouth': [-21.943, 114.134], 
                        'Carnarvon': [-24.881, 113.663], 'Geraldton': [-28.781, 114.614], 
                        'Perth': [-31.951, 115.865]}

def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a))

name_list = []
for locs in locations_to_analyse:
    locsloc = locations_to_analyse[locs]
    listloc = [locations['locLat'], locations['locLon']]    
    temp_list = []
    for num in range(len(listloc[0])):
        temp_list.append(distance(locsloc[0], locsloc[1], listloc[0][num], listloc[1][num]))
    min_dist = temp_list.index(min(temp_list))
    name_list.append(locations[min_dist])
name_list

[(10720, u'Learmonth Airport', 114.08333, -22.23333),
 (11188, u'Karratha Aerodrome Aws', 116.76667, -20.71667),
 (10789, u'Shark Bay Denham', 113.51667, -25.91667),
 (10671, u'Broome Airport', 122.21667, -17.95),
 (10789, u'Shark Bay Denham', 113.51667, -25.91667),
 (10719, u'Carnarvon Airport', 113.66667, -24.86667),
 (10728, u'Port Hedland Bedout Island Aws', 119.1, -19.58333)]

## Extract events that meet distance and speed thresholds

In [10]:
speed_thr = 70
dist_thr = 100

In [11]:
extreme_events = database.locationPassageWindSpeed(db, str(name_list[0][0]), speed_thr, dist_thr)
extreme_events

rec.array([ (u'Learmonth Airport',  72.09527588, -63.45491791,  34.22283936, u'007-00135',   6.3675266 ,  80.8937149 ,  89575.0078125),
 (u'Learmonth Airport',  76.39884949, -26.05720139,  71.81787109, u'000-00009',  14.50699698,  79.48527527,  90910.9609375)], 
          dtype=[('locName', '<U17'), ('wspd', '<f8'), ('umax', '<f8'), ('vmax', '<f8'), ('eventId', '<U9'), ('distClosest', '<f8'), ('maxwind', '<f8'), ('pmin', '<f8')])